In [1]:
from influxdb_client import InfluxDBClient
import pandas as pd
import os
from IPython.display import Markdown
import warnings

warnings.filterwarnings("ignore")

# Configura la connessione a InfluxDB
INFLUXDB_URL = "http://localhost:8086"
TOKEN = os.getenv("token")  # Assicurati che il token sia impostato correttamente
ORG = "IotAlarmSystem"
BUCKET = "Prova"

client = InfluxDBClient(url=INFLUXDB_URL, token=TOKEN, org=ORG)
query_api = client.query_api()

# Query per recuperare i dati della pressione dall'ultima settimana
query = f"""
from(bucket: "{BUCKET}")
|> range(start: -7d)
|> filter(fn: (r) => r._measurement == "pressure")
|> filter(fn: (r) => r._field == "value")
|> keep(columns: ["_time", "_value"])
"""

result = query_api.query(query)
data = []

# Converti i risultati in un DataFrame
for table in result:
    for record in table.records:
        data.append({"time": record["_time"], "value": record["_value"]})

df = pd.DataFrame(data)


df.to_csv("pressure.csv", index=False)


# Convertire la colonna "time" in formato datetime
df["time"] = pd.to_datetime(df["time"])

# Ordinare i dati per sicurezza
df = df.sort_values("time")





/Users/acco/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def calculate_total_time(df, start_time=None, end_time=None):
    """
    Funzione che prende in input un DataFrame con colonne 'time' e 'value',
    e calcola il tempo totale (in ore, minuti e secondi) sommando il tempo tra i primi 4095
    incontrati e il primo valore successivo diverso, all'interno di un periodo specificato.
    
    Se start_time e end_time non sono specificati, considera l'intero DataFrame.
    """
    import pandas as pd

    # Convertire la colonna 'time' in formato datetime con timezone
    df['time'] = pd.to_datetime(df['time'])

    # Se specificati, convertire start_time e end_time in datetime con timezone UTC
    if start_time:
        start_time = pd.to_datetime(start_time).tz_localize("UTC")
        df = df[df['time'] >= start_time]

    if end_time:
        end_time = pd.to_datetime(end_time).tz_localize("UTC")
        df = df[df['time'] <= end_time]

    # Liste per salvare i tempi di transizione e gli indici
    transitions = []
    indices_4095 = []

    i = 0
    while i < len(df):
        # Trovare il primo 4095 successivo
        while i < len(df) and df.iloc[i]['value'] != 4095:
            i += 1
        if i >= len(df):
            break

        # Salvare l'indice del 4095 selezionato
        start_index = df.index[i]  # Usare il vero indice del DataFrame
        start_time = df.iloc[i]['time']

        # Avanzare fino a trovare il primo valore diverso da 4095
        while i < len(df) and df.iloc[i]['value'] == 4095:
            i += 1
        if i >= len(df):
            break

        # Ora siamo sul primo valore diverso da 4095
        end_time = df.iloc[i]['time']
        time_diff = (end_time - start_time).total_seconds()

        # Salvare i risultati
        transitions.append(round(time_diff, 2))
        indices_4095.append(start_index)

    # Sommare tutti i secondi arrotondati
    total_seconds = sum(transitions)

    # Convertire in ore, minuti e secondi
    hours = int(total_seconds // 3600)
    minutes = int((total_seconds % 3600) // 60)
    seconds = int(total_seconds % 60)

    return f"{hours} ore, {minutes} minuti e {seconds} secondi"

# Eseguire la funzione con un periodo specificato
final_result = calculate_total_time(df)

final_result


'3 ore, 48 minuti e 7 secondi'

In [3]:
final_result = calculate_total_time(df, start_time="2025-02-01 22:42:38", end_time="2025-03-01 20:40:38")
final_result

'2 ore, 0 minuti e 1 secondi'